# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/weather_data.csv")
weather_data


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Nikolskoye,75,RU,1595758252,55,59.70,30.79,66.99,11.18
1,1,North Bend,1,US,1595758254,100,43.41,-124.22,54.00,4.70
2,2,Bilibino,7,RU,1595758255,48,68.05,166.44,72.97,7.67
3,3,Atuona,4,PF,1595758256,73,-9.80,-139.03,79.75,21.18
4,4,Punta Arenas,100,CL,1595758257,86,-53.15,-70.92,32.00,25.28
...,...,...,...,...,...,...,...,...,...,...
556,556,Amazar,93,RU,1595758982,62,53.86,120.88,77.09,8.34
557,557,Anadyr,60,RU,1595758983,82,64.75,177.48,62.60,11.18
558,558,College,40,US,1595758985,87,64.86,-147.80,53.60,4.70
559,559,Teknāf,97,BD,1595758987,73,20.86,92.31,86.25,9.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,Richards Bay,0,ZA,1595758270,22,-28.78,32.04,74.64,3.69
105,105,Urucará,0,BR,1595758388,83,-2.54,-57.76,71.26,0.25
251,251,Inzhavino,0,RU,1595758576,43,52.32,42.49,74.88,3.51
298,298,Porto Nacional,0,BR,1595758636,49,-10.71,-48.42,71.60,9.17
306,306,Aras,0,ES,1595758646,47,42.56,-2.36,78.80,5.82
308,308,Aquin,0,HT,1595758648,81,18.28,-73.40,78.60,3.09
356,356,Ponta do Sol,0,PT,1595758711,65,32.67,-17.10,72.00,2.57
388,388,Tocantins,0,BR,1595758756,49,-10.50,-48.00,71.60,9.17
418,418,Beloha,0,MG,1595758797,30,-25.17,45.05,76.28,9.46
482,482,Kununurra,0,AU,1595758882,24,-15.77,128.73,77.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,Richards Bay,ZA,-28.78,32.04,
105,Urucará,BR,-2.54,-57.76,
251,Inzhavino,RU,52.32,42.49,
298,Porto Nacional,BR,-10.71,-48.42,
306,Aras,ES,42.56,-2.36,
308,Aquin,HT,18.28,-73.40,
356,Ponta do Sol,PT,32.67,-17.10,
388,Tocantins,BR,-10.50,-48.00,
418,Beloha,MG,-25.17,45.05,
482,Kununurra,AU,-15.77,128.73,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
import time

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 14: Richards Bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 105: Urucará.
Missing field/result... skipping.
------------
Retrieving Results for Index 251: Inzhavino.
Missing field/result... skipping.
------------
Retrieving Results for Index 298: Porto Nacional.
Missing field/result... skipping.
------------
Retrieving Results for Index 306: Aras.
Missing field/result... skipping.
------------
Retrieving Results for Index 308: Aquin.
Missing field/result... skipping.
------------
Retrieving Results for Index 356: Ponta do Sol.
Missing field/result... skipping.
------------
Retrieving Results for Index 388: Tocantins.
Missing field/result... skipping.
------------
Retrieving Results for Index 418: Beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 482: Kununurra.
Missing field/result... skipping.
------------
Retrieving Results for Index 525: Menongue.
Missing field/result... skipping.
------

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))